# Gene Expression Analysis

## Environment

In [ ]:
# Loading the Packages
%reload_ext autoreload
%autoreload 2

import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc

In [ ]:
# cell-typing
from cell_typing import QC_plot, general_preprocess, preprocess_of_UMAP, UMAP_genes_plot, UMAP_obs_plot, annotate
# personalized cell-typing
from cell_typing_HCC import threshold_in_cluster, collect_liver
# spatial
from spatial import create_hull, show_cluster, ROI_mask_load
# correlation analysis
from correlation import matrix_for_heatmap

In [ ]:
BASE_DIR = Path('/PRISM_code/dataset/_example_dataset/processed')
RUN_ID = '_example_dataset'
src_dir = BASE_DIR / f'{RUN_ID}_processed'
stc_dir = src_dir / 'stitched'
read_dir = src_dir / 'readout'
seg_dir = src_dir / 'segmented'
cell_typ_dir = src_dir/"analysis_celltyping"
spatial_dir = src_dir/'analysis_spatial'
os.makedirs(cell_typ_dir, exist_ok=True)

## load exp data

In [ ]:
# load expression matrix
adata = sc.AnnData(pd.read_csv(seg_dir/"expression_matrix.csv", index_col=0))
adata.var.index = adata.var.index.str.upper()
adata.obs['dataset'] = ["PRISM3D"] * len(adata)
adata.obs['tissue'] = ['mousebrain_HP'] * len(adata)
adata.raw = adata
gene_list = adata.var.index

In [ ]:
# load spatial information
centroid = pd.read_csv(seg_dir/'dapi_predict.csv', index_col=0)
centroid_sub = centroid.loc[adata.obs.index]
adata.obsm['spatial'] = np.array([centroid_sub['y_in_pix'], centroid_sub['x_in_pix']]).T
adata.obsm['spatial3d'] = np.array([centroid_sub['x_in_pix'], 
                                    centroid_sub['y_in_pix'], 
                                    centroid_sub['z_in_pix']*3.36]).T

## Preprocess

In [ ]:
# process of gene name
adata, origin_cell_num, filtered_cell_num = general_preprocess(adata, min_genes=1, min_counts=6, max_counts=150, min_cells=1, auto_filter=False)

# copy the meta data of adata
adata_meta = adata.copy()

# direct leiden


## process

In [ ]:
adata = adata_meta.copy()
# preprocess of UMAP
adata = preprocess_of_UMAP(adata)

# compute pca
sc.tl.pca(adata)
sc.pl.pca_variance_ratio(adata, log=False)

In [ ]:
# select the num of pc
n_pcs=20
sc.tl.pca(adata, n_comps=n_pcs)                                                                               

In [ ]:
# Run UMAP
sc.pp.neighbors(adata, n_neighbors=50, n_pcs=n_pcs)
sc.tl.umap(adata)

In [ ]:
# Run Leiden cluster
sc.tl.leiden(adata, resolution=1)

## overview

### cell num of each leiden cluster

In [ ]:
a = [len(adata[adata.obs.leiden == _]) for _ in adata.obs.leiden.unique()]
fig, ax = plt.subplots(figsize=(7,3))
sns.histplot(a, bins=30, stat='count', alpha=1, kde=True,
            edgecolor='white', linewidth=0.5,
            # log=True, 
            ax=ax,
            line_kws=dict(color='black', alpha=0.7, linewidth=1.5, label='KDE'),
            # binrange=[0,100]
            )
plt.show()

adata_thre = adata[adata.obs.leiden.isin([_ for _ in adata.obs.leiden.unique() if len(adata[adata.obs.leiden == _]) > 100])]

### boxplot

In [ ]:
unknown_cluster = [str(_) for _ in sorted([int(_) for _ in adata.obs.leiden.unique()])]
fig, ax = plt.subplots(ncols=1, nrows=len(unknown_cluster),figsize =(25, 100))
for _, cluster_num  in enumerate(unknown_cluster):
    data = adata[adata.obs['leiden'] == cluster_num].X
    ax[_].boxplot(data, flierprops={'marker': 'o', 'markersize': 2, 'markerfacecolor': 'fuchsia'})
    ax[_].set_xticklabels(list(adata.var_names))
    ax[_].set_title(f'cluster{cluster_num}')
plt.show()

### umap by gene

In [ ]:
UMAP_genes_plot(adata, FOI=os.path.split(src_dir)[-1], save=False, datatype='direct', dataset=["PRISM3D"], size=3)

### umap by leiden

In [ ]:
UMAP_obs_plot(adata, FOI=os.path.split(src_dir)[-1], color='leiden', save=False, out_path='', datatype='direct')
QC_plot(adata, hue='leiden')

# harmony combine

## load sc data

In [ ]:
adata_sc = sc.read_h5ad(cell_typ_dir/'dataset_sc_rnaseq'/'sc_data_mousebrain'/'cache'/'l1_hippocampus.h5ad')
adata_sc.var.index = adata_sc.var.index.str.upper()
index_series = pd.Series(adata_sc.var.index)
index_series = index_series.replace({'3110035E14RIK': 'VXN'})
adata_sc.var.index = pd.Index(index_series)
adata_sc.var_names_make_unique()
adata_sc.obs['dataset'] = ['mousebrain_HP'] * len(adata_sc)

In [ ]:
# process of gene name
adata_sc, origin_cell_num, filtered_cell_num = general_preprocess(adata_sc, min_genes=500, min_counts=6, max_counts=150, min_cells=1, auto_filter=True)

# copy the meta data of adata
adata_sc_meta = adata_sc.copy()

In [ ]:
list_of_variable_names = adata.var_names
adata_sc_subset = adata_sc[:, list_of_variable_names]
adata_sc_subset = preprocess_of_UMAP(adata_sc_subset)

## process

In [ ]:
combine_adata = adata.concatenate(adata_sc_subset, batch_key="dataset", batch_categories=["PRISM3D", "mousebrain_HP"])

print("origin_gene_num:", len(adata.var.index))
print("combine_gene_num:", len(combine_adata.var.index))
print("Genes_not_matched:", ','.join(list(set(adata.var.index) - set(combine_adata.var.index))))

In [ ]:
sc.tl.pca(combine_adata, n_comps=29)
sc.pl.pca_variance_ratio(combine_adata, log=False)

h_pcs = 20
sc.tl.pca(combine_adata, n_comps=h_pcs)
sc_cell_num = len(combine_adata) - len(adata)
print(combine_adata)

In [ ]:
import scanpy.external as sce

sce.pp.harmony_integrate(
    combine_adata,
    "dataset",
    "X_pca",
    "X_pca_harmony",
    max_iter_harmony=30,
    # max_iter_kmeans=30,
)

In [ ]:
neighbor = 100
sc.pp.neighbors(combine_adata, n_neighbors=neighbor, use_rep="X_pca_harmony")
sc.tl.umap(combine_adata)

In [ ]:
leiden_resolution=1
sc.tl.leiden(combine_adata, resolution=leiden_resolution)

## overview

### cell num of each leiden cluster

In [ ]:
combine_adata_exp = combine_adata[combine_adata.obs.dataset == 'PRISM3D']
a = [len(combine_adata_exp[combine_adata_exp.obs.leiden == _]) for _ in combine_adata_exp.obs.leiden.unique()]
fig, ax = plt.subplots(figsize=(7,3))
sns.histplot(a, bins=30, stat='count', 
             alpha=1, edgecolor='white', linewidth=0.5, ax=ax,
             kde=True, line_kws=dict(color='black', alpha=0.7, linewidth=1.5, label='KDE'),
            )
plt.show()

combine_adata_exp_thre = combine_adata_exp[combine_adata_exp.obs.leiden.isin([_ for _ in combine_adata_exp.obs.leiden.unique() if len(combine_adata_exp[combine_adata_exp.obs.leiden == _]) > 100])]

### boxplot

In [ ]:
unknown_cluster = [str(_) for _ in sorted([int(_) for _ in combine_adata.obs.leiden.unique()])]
fig, ax = plt.subplots(ncols=1, nrows=len(unknown_cluster),figsize =(25, 100))
for _, cluster_num  in enumerate(unknown_cluster):
    data = combine_adata[combine_adata.obs['leiden'] == cluster_num].X
    ax[_].boxplot(data, flierprops={'marker': 'o', 'markersize': 2, 'markerfacecolor': 'fuchsia'})
    ax[_].set_xticklabels(list(combine_adata.var_names))
    ax[_].set_title(f'cluster{cluster_num}')
plt.show()

### umap by gene

In [ ]:
UMAP_genes_plot(combine_adata, FOI=os.path.split(src_dir)[-1], save=False, datatype='harmony', dataset=['PRISM3D'], size=2)

### umap by leiden

In [ ]:
UMAP_obs_plot(combine_adata, FOI=os.path.split(src_dir)[-1], color='leiden', save=False, out_path=cell_typ_dir, datatype='harmony', DOI=['PRISM3D', 'mousebrain_HP'], size=5)

## annotate interested clusters

### relabel mapping

In [ ]:
cluster_dict = {
    "Ex-CA1":[7],
    "Ex-CA2":[12,],
    "Ex-CA3":[11],
    "Ex-DG":[3],

    "Ex-MH":[9],
    "Ex-PMCH+":[22],
    "Ex-thalamus":[15], 

    "Glial-Microglia":[13],
    "Glial-Astrocyte":[5],
    "Glial-Oligodendrocyte":[0],
    
    "In-Pvalb":[18],
    "In-Sst":[8],
    "In-Vip":[16],
    "In_Lamp5":[6], 
}

cluster_rough_dict = {
    "Ex":[],
    "In":[],
    "Glial":[],    
}
for rough_type in cluster_rough_dict.keys():
    for fine_type in cluster_dict.keys():
        if rough_type in fine_type:
            cluster_rough_dict[rough_type] += cluster_dict[fine_type]

### spatial preview

In [ ]:
combine_adata_st = combine_adata[combine_adata.obs.dataset == 'PRISM3D']
combine_adata_st.obs.index = [int(_.split('-')[0]) for _ in combine_adata_st.obs.index]
centroid = pd.read_csv(os.path.join(seg_dir, 'dapi_predict.csv'), index_col=0)
centroid_sub = centroid.loc[combine_adata_st.obs.index]
combine_adata_st.obsm['spatial'] = np.array([centroid_sub['y_in_pix'], 
                                            centroid_sub['x_in_pix']]).T
combine_adata_st.obsm['spatial3d'] = np.array([centroid_sub['x_in_pix'], 
                                               centroid_sub['y_in_pix'], 
                                               centroid_sub['z_in_pix']*3.36]).T

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.gridspec as gridspec

group = 0

fig = plt.figure(figsize=(12, 5))
gs = gridspec.GridSpec(1, 10)

ax_2d = fig.add_subplot(gs[0, :5])
ax_2d.invert_yaxis()
sc.pl.embedding(combine_adata_st[combine_adata_st.obs.leiden == str(group)], 
                basis="spatial", color="leiden", palette=['brown'],
                size=30, alpha=1, ax=ax_2d, show=False)

ax_3d = fig.add_subplot(gs[0, 5:], projection='3d', )
sc.pl.embedding(combine_adata_st[combine_adata_st.obs.leiden == str(group)], 
                basis="spatial3d", projection="3d", color="leiden", palette=['brown'],
                size=1, alpha=1, ax=ax_3d, show=False)

# Set view angle
elev, azim = 90, 0
ax_3d.view_init(elev=elev, azim=azim)

plt.tight_layout()
plt.show()

### relabel of num and cluster name

In [ ]:
# rename leidens based on cluster_dict
combine_adata_st = annotate(combine_adata_st, cluster_dict, in_leiden='leiden', out_leiden='leiden_subtype', out_type='subtype')
combine_adata_st = annotate(combine_adata_st, cluster_rough_dict, in_leiden='leiden', out_leiden='leiden_type', out_type='type')
combine_adata_st.obs = combine_adata_st.obs.dropna(axis=1)

In [ ]:
for cell_type in cluster_dict.keys():
    print(f'{cell_type}: {combine_adata_st[combine_adata_st.obs.subtype == cell_type].obs.subtype.count()}')

print('total_exempt_other:',len(combine_adata_st[combine_adata_st.obs.subtype.isin(cluster_dict.keys())]))
# print('total_exempt_other:',len(adata[adata.obs.type!='other']))
print('total:',len(combine_adata_st))

# Plot1: umap and projection after threshold


## dotplot for type and subtype

In [ ]:
adata_st_reordered = sc.read_h5ad(cell_typ_dir/'adata.h5ad')
adata_raw_reordered = sc.read_h5ad(seg_dir/'adata_reordered.h5ad')

adata_tmp = adata_raw_reordered.copy()
adata_tmp.obs = adata_st_reordered.obs.copy()
adata_tmp.raw = None  # remember that raw counts will be used to plot dotplots when it exists.

In [ ]:
tmp_var_names = [
    'HBV', 'AFP', 'GPC3', 'MKI67', 'PECAM1', 'EPCAM', 'ACTA2',
    'FOXP3',
    'CD3D', 'CD4', 'PDCD1', 'CTLA4', 'CXCL13', 'NCAM1', 'GZMA', 'GZMB', 'PRF1', 'CD8A',
    'CD79A', 'MS4A1',
    'LILRA4',
    'CLEC9A', 'CD1C', 'LYVE1',
    'C1QA', 'FCGR3A', 'S100A8', 'CSF3R',
    'MZB1', 'SLC4A10', 'CPA3']


tmp_category = [
    'Liver','Tumor','Endo','Ep','CAF',
    'T_reg','T_proliferation','NK','CD4+','CD8+','B',
    'DC','Macrophage','Monocyte','Neutrophil','Mait','Mast']

tmp = adata_tmp[adata_tmp.obs.type != 'other']
tmp.obs.type = pd.Categorical(tmp.obs.type, categories=tmp_category)

fig, ax = plt.subplots(figsize=(12, 10))
sc.pl.dotplot(tmp,var_names=tmp_var_names,
              groupby='type',vmax=5,ax=ax,show=False)
plt.show()
# plt.savefig(cell_typ_dir/r'1_dotplot_type.pdf', bbox_inches = 'tight')
# plt.savefig(cell_typ_dir/r'1_dotplot_type.png', bbox_inches = 'tight', dpi=300)

In [ ]:
tmp_var_names = [
    'HBV', 'AFP', 'GPC3', 'MKI67', 'PECAM1', 'EPCAM', 'ACTA2',
    'FOXP3',
    'CD3D', 'CD4', 'CTLA4', 'PDCD1', 'CXCL13',
    'CD8A', 'NCAM1', 'GZMA', 'GZMB', 'PRF1',
    'CD79A', 'MS4A1',
    'LILRA4',
    'CLEC9A', 'CD1C', 'LYVE1',
    'C1QA', 'FCGR3A', 'S100A8', 'CSF3R',
    'MZB1', 'SLC4A10', 'CPA3']

tmp_category = [
    'Liver_normal', 'Tumor_AFP+', 'Tumor_GPC3+', 'Tumor_proliferation',
    'Endo_PECAM1+', 'Ep_EPCAM+', 'CAF_ACTA2+',
    'T_reg', 'T_proliferation',
    "T_CD4+_other", "T_CD4+, CTLA4+", "T_CD4+, PD1+, CTLA4+", "T_CD4+, PD1+",
    # "T_CD4+, PD1+, CXCL13+",
    "T_CD4+, CXCL13+",
    "T_CD8+, GZMA+, CXCL13+", "T_CD8+, PD1+", "T_CD8+_other",
    "Cyto_T_CD4+", 'NK_NCAM1+',
    'B_CD79A+', 'B_MS4A1+',
    # 'B_CD79A+, MS4A1+',
    'pDC_LILRA4+', 'cDC1_CLEC9A+', 'cDC2_CD1C+',
    # 'Macrophage_C1QA+',
    'Macrophage_LYVE1+',
    'Monocyte_CD16+', 'Monocyte_CD14+, CD16+', 'Monocyte_CD14+',
    'Neutrophil_CSF3R+, S100A8+', 'Neutrophil_CSF3R+',
    'Mait_SLC4A10+', 'Mast_CPA3+'
]

tmp = adata_tmp[adata_tmp.obs.subtype != 'other']
tmp.obs.subtype = tmp.obs.subtype.replace(
    "T_CD4+, PD1+, CXCL13+", "T_CD4+, CXCL13+")

tmp.obs.subtype = pd.Categorical(tmp.obs.subtype, categories=tmp_category)

fig, ax = plt.subplots(figsize=(14, 12))
sc.pl.dotplot(tmp, var_names=tmp_var_names,
              groupby='subtype', vmax=5, ax=ax, show=False)
plt.show()
# plt.savefig(cell_typ_dir/r'1_dotplot_subtype.pdf', bbox_inches='tight')
# plt.savefig(cell_typ_dir/r'1_dotplot_subtype.png', bbox_inches='tight', dpi=300)

## umap colored by genes, cluster and dataset

In [ ]:
type_colormap = {
    'Liver':(1,0.392,0),
    'Tumor':(0.751,0.491,0),
    'Endo':(1,0,1),
    'Ep':(0,1,0),
    'CAF':(0,0,1),
    'DC':(1,0.259,0),
    'Mait':(1,0,0.434),
    'Mast':(1,0,0),
    'Monocyte':(0,0.471,1),
    'Neutrophil':(1,1,0),
    'Macrophage':(0.7,1,0),
    'CD4+':(0.5,0.5,0.5),
    'CD8+':(1,0.8,0),
    'T_reg':(0,1,0.672),
    'T_proliferation':(0,1,0.636),
    'B':(0,1,1),
    'NK':(1,0,0),
    'other':(0.9,0.9,0.9),
}

subtype_colormap = dict()
for subtype in cluster_dict.keys():
    for rough_type in type_colormap.keys():
        if rough_type in subtype:
            subtype_colormap[subtype] = type_colormap[rough_type]
            break

In [ ]:
tmp = combine_adata_st[~combine_adata_st.obs['tmp_leiden'].isin(['-2', '-1'])]
UMAP_obs_plot(adata=tmp, color='type', palette=type_colormap,
              save=True, out_path=cell_typ_dir,
              dpi=300, datatype='harmony', legend_loc='right margin')

# UMAP_genes_plot(adata=tmp,
# FOI=FOI, save=False, out_path=r'./', datatype='harmony', dataset=['PRISM_HCC'])

## plot gene distribution of each cluster

In [ ]:
for cluster_num in [str(_) for _ in []]:
    cluster=combine_adata_st[combine_adata_st.obs['leiden']==cluster_num].X
    fig, ax = plt.subplots(figsize=(20, 4))
    plt.boxplot(cluster, labels=combine_adata_st.var_names)
    fig.tight_layout()
    fig.suptitle(f'distribution of cluster{cluster_num}')
    plt.show()

# Plot2: spatial projection

## 3d projection

In [ ]:
import matplotlib.pyplot as plt
# import numpy as np
from skimage import filters
from PIL import Image
import io
import gc

step = 255//len(cluster_rough_dict)
colors = [(1,1,1) for _ in range(step, 256, step)]

cluster_image = {}
for _ in cluster_rough_dict.keys():
    cluster_image[_] = np.zeros([10860, 12480, 20], dtype=bool)

for layer_num in range(20):
    print(f'layer{layer_num+1}')
    tmp_adata = combine_adata_st[combine_adata_st.obs['layer']==f'layer{layer_num}']
    tmp_adata = tmp_adata[tmp_adata.obs.tmp_leiden != '-2']
    tmp_adata.obs.index = [_.split('-')[0] for _ in tmp_adata.obs.index]

    hulls, type_indices = create_hull(tmp_adata, clus_obs="leiden_type", cont_thre=7,
                                        rna_pos = pd.read_csv(f"E:\TMC\cell_typing\exp_dataset\PRISM_HCC_of_20_slides\S_{layer_num+1}_rna_labeled.csv"))

    for _, cluster_name in enumerate(cluster_rough_dict.keys()):
        # Create a Matplotlib figure
        fig, ax = plt.subplots()
        show_cluster(hulls, type_indices, cluster_list=[_], cluster_colormap=colors, linewidth=0.3, 
                    ax=ax,
                    show=False , save=False, 
                    )

        # Convert the colorful plot to grayscale
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        # Remove the title
        ax.set_title('')
        # Remove the axis labels
        ax.set_xlabel('')
        ax.set_ylabel('')
        # Remove the grid lines
        # ax.grid(False)
        ax.axis('off')

        # Set the figure to use 8-bit grayscale
        fig.set_facecolor('black')
        fig.set_figwidth(40)
        fig.set_figheight(35)
        fig.set_dpi(1000)

        # Render the figure to a PIL Image
        buf = io.BytesIO()
        fig.savefig(buf, dpi=400, bbox_inches='tight', format='png')
        buf.seek(0)
        
        img = Image.open(buf).convert('L')  # Convert to grayscale
        img_array = np.array(img)
        thresholded_array = img_array >= 128
        
        cluster_image[cluster_name][:, :, _] = thresholded_array

        plt.close(fig=fig)       
        img.close()
        buf.close()

In [ ]:
import tifffile as tiff
for a, _ in enumerate(cluster_image.keys()):
    tmp = np.where(cluster_image[_], 255, 0).astype(np.uint8)
    tiff.imwrite(f"E:/TMC/cell_typing/results/2023.9.19-9.22_PRISM_HCC_20_layers_min_counts=7, max_counts=200, min_genes=2/projection/trial3/channel{a}_{_}.tif", np.transpose(tmp, (2, 0, 1)))

## 2d projection

In [ ]:
# plot type cluster
for layer_num in range(20):
    tmp_adata = combine_adata_st[combine_adata_st.obs['layer']
                                 == f'layer{layer_num}']
    tmp_adata = tmp_adata[tmp_adata.obs.tmp_leiden != '-2']
    tmp_adata.obs.index = [_.split('-')[0] for _ in tmp_adata.obs.index]
    hulls, type_indices = create_hull(tmp_adata, clus_obs="leiden_type", cont_thre=7,
                                      rna_pos=pd.read_csv(f"E:\TMC\cell_typing\exp_dataset\PRISM_HCC_of_20_slides\S_{layer_num+1}_rna_labeled.csv"))

    ncols = int(-(-len(cluster_rough_dict)**(1/2)//1))
    nrows = -(-len(cluster_rough_dict)//ncols)
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols,
                           figsize=(ncols*4, nrows*4))
    for cluster_num, cluster_type in enumerate(cluster_rough_dict.keys()):
        show_cluster(hulls, type_indices, cluster_list=[cluster_num], cluster_colormap=["red"] * 200, linewidth=0.3,
                     ax=ax[cluster_num // ncols][cluster_num % ncols],
                     show=False, save=False, name=cluster_type)
        ax[cluster_num // ncols][cluster_num % ncols].set_xlabel("")
        ax[cluster_num // ncols][cluster_num % ncols].set_ylabel("")
        ax[cluster_num // ncols][cluster_num % ncols].set_xticks([])
        ax[cluster_num // ncols][cluster_num % ncols].set_yticks([])
    fig.suptitle(
        f'Projection_of_layer{layer_num+1}, cell_num={len(tmp_adata)}\n\n', fontsize=20)
    plt.tight_layout()
    plt.savefig(
        f'e:\TMC\cell_typing\results\2023.9.19-9.22_PRISM_HCC_20_layers_min_counts=7, max_counts=200, min_genes=2\projection\{cluster_num}_{cluster_type}_projection.png')
    # plt.show()

In [ ]:
# plot subtype cluster
for layer_num in range(20):
    tmp_adata = combine_adata_st[combine_adata_st.obs['layer']
                                 == f'layer{layer_num}']
    tmp_adata = tmp_adata[tmp_adata.obs.tmp_leiden != '-2']
    tmp_adata.obs.index = [_.split('-')[0] for _ in tmp_adata.obs.index]
    hulls, type_indices = create_hull(tmp_adata, clus_obs="leiden_subtype", cont_thre=7,
                                      rna_pos=pd.read_csv(f"E:\TMC\cell_typing\exp_dataset\PRISM_HCC_of_20_slides\S_{layer_num+1}_rna_labeled.csv"))

    ncols = int(-(-len(cluster_dict)**(1/2)//1))
    nrows = -(-len(cluster_dict)//ncols)
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols,
                           figsize=(ncols*4, nrows*4))
    for cluster_num, cluster_type in enumerate(cluster_dict.keys()):
        show_cluster(hulls, type_indices, cluster_list=[cluster_num], cluster_colormap=["red"] * 200, linewidth=0.3,
                     ax=ax[cluster_num // ncols][cluster_num % ncols],
                     show=False, save=False, name=cluster_type)
        ax[cluster_num // ncols][cluster_num % ncols].set_xlabel("")
        ax[cluster_num // ncols][cluster_num % ncols].set_ylabel("")
        ax[cluster_num // ncols][cluster_num % ncols].set_xticks([])
        ax[cluster_num // ncols][cluster_num % ncols].set_yticks([])
    fig.suptitle(
        f'Projection_of_layer{layer_num+1}, cell_num={len(tmp_adata)}\n\n', fontsize=20)
    plt.tight_layout()
    plt.show()

## subsample of AFP

In [ ]:
adata_tumor_plot_AFP_subsample = sc.pp.subsample(
    combine_adata_st[combine_adata_st.obs.subtype == 'Tumor_AFP+'], n_obs=45730, copy=True)


adata_tumor_plot_GPC3 = combine_adata_st[combine_adata_st.obs.subtype == 'Tumor_GPC3+']

adata_tumor_plot = combine_adata_st[combine_adata_st.obs.index.isin(list(
    adata_tumor_plot_AFP_subsample.obs.index)+list(adata_tumor_plot_GPC3.obs.index))]

adata_tumor_plot.write(
    spatial_dir/r'2023.10.6_adata_AFPsubsampled_and_GPC3+.h5ad')

In [ ]:
from collections import Counter
before_filter = Counter(list(adata_tumor_plot_AFP_subsample.obs['layer']))
after_filter = Counter(list(adata_tumor_plot_GPC3.obs['layer']))

data = dict(sorted(before_filter.items(), key=lambda d: int(
    d[0].replace('layer', '')), reverse=False))
courses = list(data.keys())
values = list(data.values())
fig = plt.figure(figsize=(15, 3))
# creating the bar plot
plt.bar(courses, values)

data = dict(sorted(after_filter.items(), key=lambda d: int(
    d[0].replace('layer', '')), reverse=False))
courses = list(data.keys())
values = list(data.values())
fig = plt.figure(figsize=(15, 3))
# creating the bar plot
plt.bar(courses, values)

courses = list(data.keys())
values = [list(dict(sorted(before_filter.items(), key=lambda d: int(d[0].replace('layer', '')), reverse=False)).values())[_]/list(dict(sorted(after_filter.items(), key=lambda d: int(
    d[0].replace('layer', '')), reverse=False)).values())[_] for _ in range(len(list(dict(sorted(after_filter.items(), key=lambda d: int(d[0].replace('layer', '')), reverse=False)).values())))]
fig = plt.figure(figsize=(15, 3))
# creating the bar plot
plt.bar(courses, values)

## density of cell

In [ ]:
# 48000, 40000
adata_tumor_plot_GPC3 = combine_adata_st[combine_adata_st.obs.subtype == 'Tumor_GPC3+']
adata_tumor_plot_AFP = combine_adata_st[combine_adata_st.obs.subtype == 'Tumor_AFP+']

downsample = 200

for layer in tqdm(range(20), desc=f'layer'):
    images = {'AFP':np.zeros((40000//downsample, 48000//downsample), dtype=np.uint8), 'GPC3':np.zeros((40000//downsample, 48000//downsample), dtype=np.uint8)}

    tmp = adata_tumor_plot_AFP[adata_tumor_plot_AFP.obs.layer == f'layer{layer}']
    for cell in tmp.obs.index:
        x = max(0, int(tmp.obs.X_pos.loc[cell]/downsample))
        y = max(0, int(tmp.obs.Y_pos.loc[cell]/downsample))
        images['AFP'][y, x] += 1

    tmp = adata_tumor_plot_GPC3[adata_tumor_plot_GPC3.obs.layer == f'layer{layer}']
    for cell in tmp.obs.index:
        x = max(0, int(tmp.obs.X_pos.loc[cell]/downsample))
        y = max(0, int(tmp.obs.Y_pos.loc[cell]/downsample))
        images['GPC3'][y, x] += 1

    for celltype in images.keys():
        io.imsave(spatial_dir/rf'2023.10.6_AFPsubsample_downsize_layer{layer + 1}_{celltype}.tif',
                images[celltype].astype(np.uint8))

## spatial projection by HBV

In [ ]:
combine_adata_st.obs['HBV_content'] = [0]*len(combine_adata_st)

for layer in range(20):
    tmp1 = combine_adata_st[combine_adata_st.obs.layer == f'layer{layer}']
    tmp1_index = tmp1.obs.index
    tmp1.obs.index = [_.split('-')[0] for _ in tmp1.obs.index]

    tmp2 = adata[adata.obs.layer == f'layer{layer}']
    tmp2.obs.index = [_.split('-')[0] for _ in tmp2.obs.index]

    tmp2 = tmp2[tmp1.obs.index]

    combine_adata_st.obs['HBV_content'][tmp1_index] = pd.Series([_[0] for _ in tmp2[:,'HBV'].X],index=tmp1_index)

In [ ]:
from scipy.signal import argrelextrema

fig, ax = plt.subplots(ncols=1,nrows=1,figsize=(20,5))
a=combine_adata_st[combine_adata_st.obs.HBV_content!=0].obs['HBV_content']
sns.histplot(a, bins=100, stat='count', alpha=1, kde=True,
            edgecolor='white', linewidth=0.5,
            # log=True,
            ax=ax,
            line_kws=dict(color='black', alpha=0.7, linewidth=1.5, label='KDE'),
            # binrange=[0,100]
            )
y=ax.get_lines()[0].get_ydata()
maxima = [float(j/len(y)*(max(a)-min(a))+min(a)) for j in argrelextrema(-np.array(y), np.less)[0]]

for submaxima in maxima:
    ax.axvline(x=submaxima, color='r', alpha=0.5, linestyle='--')
    
plt.tight_layout()
plt.show()

### devide by counts

In [ ]:
HBV_grade = {
    0: 1,
    1: 2,
    2: 3,
    3: 4,
    4: 5,
    5: 6,
    6: 7,
    7: 8,
    8: 9,
    9: 10,
    10: 20,
    11: 30,
    12: 40,
    13: 50,
    14: 60,
}

# HBV_grade = sorted(HBV_grade.items(), key=lambda x: -x[1])

# combine_adata_st.obs.leiden['HBV_grade'] = pd.Categorical([0]*len(combine_adata_st), categories=list(ROI_mask.keys()) + ['other'], ordered=False)
combine_adata_st.obs["HBV_grade"] = [-2] * len(combine_adata_st)
for cell in tqdm(combine_adata_st.obs.index):
    for grade, value in HBV_grade.items():
        if combine_adata_st.obs['HBV_content'].loc[cell] >= value:
            combine_adata_st.obs['HBV_grade'].loc[cell] = grade

### devide by percentile

In [ ]:
tmp = combine_adata_st[combine_adata_st.obs.HBV_content != 0]
# tmp = tmp[tmp.obs.tmp_leiden != '-2']
content = list(tmp.obs.HBV_content)
np.percentile(content, [34,57,78])

In [ ]:
HBV_grade = {
    0: 1,
    1: 2,
    2: 4,
}

# HBV_grade = sorted(HBV_grade.items(), key=lambda x: -x[1])

# combine_adata_st.obs.leiden['HBV_grade'] = pd.Categorical([0]*len(combine_adata_st), categories=list(ROI_mask.keys()) + ['other'], ordered=False)
combine_adata_st.obs["HBV_grade_4"] = [-2] * len(combine_adata_st)
for cell in tqdm(combine_adata_st.obs.index):
    for grade, value in HBV_grade.items():
        if combine_adata_st.obs['HBV_content'].loc[cell] >= value:
            combine_adata_st.obs['HBV_grade_4'].loc[cell] = grade

### devide by detailed percentile

In [ ]:
tmp = combine_adata_st[combine_adata_st.obs.HBV_content != 0]
# tmp = tmp[tmp.obs.tmp_leiden != '-2']
content = list(tmp.obs.HBV_content)
content = sorted(content)
# np.percentile(content, [34,57,74,83,85,90,93,96,97,98,98.5,99])

In [ ]:
for i in range(20):
    print(i+1, content.index(i+1)/len(content))

In [ ]:
HBV_grade = {
    0: 1,
    1: 2,
    2: 3,
    3: 4,
    4: 5,
    5: 6,
    6: 7,
    7: 8,
    8: 11,
}

# HBV_grade = sorted(HBV_grade.items(), key=lambda x: -x[1])

# combine_adata_st.obs.leiden['HBV_grade'] = pd.Categorical([0]*len(combine_adata_st), categories=list(ROI_mask.keys()) + ['other'], ordered=False)
combine_adata_st.obs["HBV_grade_detailed"] = [-2] * len(combine_adata_st)
for cell in tqdm(combine_adata_st.obs.index):
    for grade, value in HBV_grade.items():
        if combine_adata_st.obs['HBV_content'].loc[cell] >= value:
            combine_adata_st.obs['HBV_grade_detailed'].loc[cell] = grade

## spatial projection by AFP

In [ ]:
combine_adata_st.obs['AFP_content'] = [0]*len(combine_adata_st)

for layer in range(20):
    tmp1 = combine_adata_st[combine_adata_st.obs.layer == f'layer{layer}']
    tmp1_index = tmp1.obs.index
    tmp1.obs.index = [_.split('-')[0] for _ in tmp1.obs.index]

    tmp2 = adata[adata.obs.layer == f'layer{layer}']
    tmp2.obs.index = [_.split('-')[0] for _ in tmp2.obs.index]

    tmp2 = tmp2[tmp1.obs.index]

    combine_adata_st.obs['AFP_content'][tmp1_index] = pd.Series([_[0] for _ in tmp2[:,'AFP'].X],index=tmp1_index)

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(20, 5))
a = combine_adata_st[combine_adata_st.obs.AFP_content != 0].obs['AFP_content']
sns.histplot(a, bins=100, stat='count', alpha=1, kde=True,
             edgecolor='white', linewidth=0.5,
             # log=True,
             ax=ax, line_kws=dict(color='black', alpha=0.7,
                                  linewidth=1.5, label='KDE'),
             # binrange=[0,100]
             )
y = ax.get_lines()[0].get_ydata()
maxima = [float(j/len(y)*(max(a)-min(a))+min(a))
          for j in argrelextrema(-np.array(y), np.less)[0]]

for submaxima in maxima:
    ax.axvline(x=submaxima, color='r', alpha=0.5, linestyle='--')

plt.tight_layout()
plt.show()

### devide by percentile

In [ ]:
tmp = combine_adata_st[combine_adata_st.obs.AFP_content != 0]
# tmp = tmp[tmp.obs.tmp_leiden != '-2']
content = sorted(list(tmp.obs.AFP_content))

In [ ]:
for i in range(20):
    print(i+1, content.index(i+1)/len(content))

In [ ]:
HBV_grade = {
    0: 1,
    1: 2,
    2: 3,
    3: 4,
    4: 5,
    5: 6,
    6: 7,
    7: 9,
    8: 11,
    9: 15,
    10: 20,
}

# HBV_grade = sorted(HBV_grade.items(), key=lambda x: -x[1])

# combine_adata_st.obs.leiden['HBV_grade'] = pd.Categorical([0]*len(combine_adata_st), categories=list(ROI_mask.keys()) + ['other'], ordered=False)
combine_adata_st.obs["AFP_grade"] = [-2] * len(combine_adata_st)
for cell in tqdm(combine_adata_st.obs.index):
    for grade, value in HBV_grade.items():
        if combine_adata_st.obs['AFP_content'].loc[cell] >= value:
            combine_adata_st.obs['AFP_grade'].loc[cell] = grade

In [ ]:
combine_adata_st.write(spatial_dir/'2023.10.12_adata_AFP_content.h5ad')

### devide by detailed percentile

In [ ]:
tmp = combine_adata_st[combine_adata_st.obs.HBV_content != 0]
# tmp = tmp[tmp.obs.tmp_leiden != '-2']
content = list(tmp.obs.HBV_content)
content = sorted(content)
# np.percentile(content, [34,57,74,83,85,90,93,96,97,98,98.5,99])

In [ ]:
for i in range(20):
    print(i+1, content.index(i+1)/len(content))

In [ ]:
HBV_grade = {
    0: 1,
    1: 2,
    2: 3,
    3: 4,
    4: 5,
    5: 6,
    6: 7,
    7: 8,
    8: 11,
}
# HBV_grade = sorted(HBV_grade.items(), key=lambda x: -x[1])
# combine_adata_st.obs.leiden['HBV_grade'] = pd.Categorical([0]*len(combine_adata_st), categories=list(ROI_mask.keys()) + ['other'], ordered=False)
combine_adata_st.obs["HBV_grade_detailed"] = [-2] * len(combine_adata_st)
for cell in tqdm(combine_adata_st.obs.index):
    for grade, value in HBV_grade.items():
        if combine_adata_st.obs['HBV_content'].loc[cell] >= value:
            combine_adata_st.obs['HBV_grade_detailed'].loc[cell] = grade

# Plot3: cluster distribution of different regions

## region of type

In [ ]:
Region_mask = ROI_mask_load(input_path=spatial_dir/'area_cloud_for_analysis', save=False, blur=False,
                            out_path=spatial_dir/'downstream_analysis')

In [ ]:
from collections import Counter


combine_adata_st.obs['region'] = pd.Categorical(['other']*len(combine_adata_st), categories=list(Region_mask.keys()) + ['other'], ordered=False)    
for _, mask in Region_mask.items():
    yrange = mask.shape[1]
    for cell in tqdm(combine_adata_st[combine_adata_st.obs['leiden_type']!='-2'].obs.index, desc=_):
        y = yrange - max(1,int(float(combine_adata_st.obs['Y_pos'].loc[cell])/100))
        x = int(max(0,float(combine_adata_st.obs['X_pos'].loc[cell])/100))
        z = int(combine_adata_st.obs['layer'].loc[cell].replace('layer',''))
        # z = int(combine_adata_st[cell].obsm['spatial'][0][2]/10*0.1625/10*0.1625)
        if mask[z, y, x]:
            combine_adata_st.obs['region'].loc[cell] = _
    
    tmp_adata_st = combine_adata_st[combine_adata_st.obs.region == _]
    tmp = Counter(tmp_adata_st[tmp_adata_st.obs.leiden_type != '-2'].obs.type)
    df = pd.DataFrame(tmp, index=[0]).T
    df.columns = [_]
    Region_cluster = pd.concat([Region_cluster, df], axis=1)

In [ ]:
Region_cluster = pd.DataFrame(index=list(cluster_rough_dict.keys()))
for _, mask in Region_mask.items():
    tmp_adata_st = combine_adata_st[combine_adata_st.obs.region == _]
    tmp = Counter(tmp_adata_st[tmp_adata_st.obs.leiden_type != '-2'].obs.type)
    df = pd.DataFrame(tmp, index=[0]).T
    df.columns = [_]
    Region_cluster = pd.concat([Region_cluster, df], axis=1)

In [ ]:
Region_cluster_immune = Region_cluster[Region_cluster.index.isin(['Liver','Tumor','Endo','Ep','CAF'])]
Region_cluster_nonimmune = Region_cluster[~Region_cluster.index.isin(['Liver','Tumor','Endo','Ep','CAF'])]

In [ ]:
import matplotlib.ticker as mtick
from matplotlib import cm
from matplotlib.colors import LinearSegmentedColormap

fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(50, 10))
# sc.pl.umap(
#     combine_adata_st[~combine_adata_st.obs['leiden_type'].isin(['-2','0'])],
#     size=1,
#     color="type",
#     palette=type_colormap,
#     # legend_loc="on data",
#     legend_fontweight=100,
#     legend_fontsize=10,
#     ax=ax[0],
#     show=False,
# )


colors = [_ for _ in type_colormap.values()][:-1]
cmap1 = LinearSegmentedColormap.from_list('my_colormap', colors[:5])
cmap2 = LinearSegmentedColormap.from_list('my_colormap', colors[5:])

df = Region_cluster_immune.T
# df = df.drop('Liver', axis=1)
df = df.div(df.sum(axis=1), axis=0) * 100
df.plot(kind='bar', 
        stacked=True, 
        figsize=(20,10), 
        colormap=cmap1,
        ax=ax[0]
        )
ax[0].yaxis.set_major_formatter(mtick.PercentFormatter())
ax[0].set_title('content of different cluster on ROIs')
# plt.legend(loc='upper center',
#            bbox_to_anchor=(1.2, 1),
#            ncols=1,
#            )

df = Region_cluster_nonimmune.T
# df = df.drop('Liver', axis=1)
df = df.div(df.sum(axis=1), axis=0) * 100
df.plot(kind='bar', 
        stacked=True, 
        figsize=(20,10), 
        colormap=cmap2,
        ax=ax[1]
        )
ax[1].yaxis.set_major_formatter(mtick.PercentFormatter())
ax[1].set_title('content of different cluster on ROIs')
# plt.legend(loc='upper center',
#            bbox_to_anchor=(1.2, 1),
#            ncols=1,
#            )
# plt.tight_layout()
# plt.show()
plt.savefig(r'E:\TMC\cell_typing\results\2023.9.28-_PRISM_HCC_final_downstream_analysis\figures\5_region_fraction.pdf')

## ROI

In [ ]:
combine_adata_st = sc.read_h5ad(spatial_dir/'adata_neighborhood_enrichment.h5ad')
adata = sc.read_h5ad(spatial_dir/'adata_relayered.h5ad')

In [ ]:
ROI_mask = ROI_mask_load(input_path=r'E:\TMC\cell_typing\exp_dataset\PRISM_HCC_of_20_slides\psuedo3D_HCC_1_ROI', save=False, out_path=r'E:\TMC\cell_typing\results\2023.9.28-_PRISM_HCC_20layers_downstream_analysis')

### ROI of gene

In [ ]:
tmp = combine_adata_st[~combine_adata_st.obs.type.isin(['other'])]
tmp = tmp[tmp.obs.ROI == 'ROI_1']

In [ ]:
ROI_dict = pd.DataFrame(index=[f'ROI_{x+1}' for x in range(5)], columns=gene_list)
for ROI in [f'ROI_{x+1}' for x in range(5)]:
    ROI_of_gene_toplot = pd.DataFrame(index=[f'layer{x}' for x in range(20)],
                                      columns=['cell_num']+gene_list)
    for layer in tqdm(range(20), desc=ROI):
        tmp1 = combine_adata_st[combine_adata_st.obs.layer == f'layer{layer}']
        tmp1 = tmp1[tmp1.obs.ROI == ROI]
        tmp1_index = tmp1.obs.index
        tmp1.obs.index = [_.split('-')[0] for _ in tmp1.obs.index]

        tmp2 = adata[adata.obs.layer == f'layer{layer}']
        tmp2.obs.index = [_.split('-')[0] for _ in tmp2.obs.index]
        tmp2 = tmp2[tmp1.obs.index]
        
        ROI_of_gene_toplot.loc[f'layer{layer}',:] = [len(tmp1)] + [np.sum([_[0] for _ in tmp2[:,gene].X]) for gene in gene_list]
    
    
    ROI_dict.loc[ROI,:] = np.sum(ROI_of_gene_toplot[gene_list], axis=0)
                                #  weights=ROI_of_gene_toplot['cell_num'], 

In [ ]:
df = ROI_dict.div(ROI_dict.sum(axis=1), axis=0) * 100
# df = ROI_dict.copy()
df[['HBV', 'AFP', 'GPC3', 'MKI67']].plot(kind='bar', 
        stacked=True, 
        figsize=(8,10), 
        # colormap=cmap1,
        # ax=ax[0]
        title='counts of different genes on ROIs'
        )
# ax[0].yaxis.set_major_formatter(mtick.PercentFormatter())
# ax[0].set_title('content of different cluster on ROIs')

### ROI of type

In [ ]:
combine_adata_st.obs['X_pos'] = [0]*len(combine_adata_st)
combine_adata_st.obs['Y_pos'] = [0]*len(combine_adata_st)

for layer in range(20):
    centroids = pd.read_csv(f'E:\TMC\cell_typing\exp_dataset\PRISM_HCC_of_20_slides\S_{layer+1}_centroids.csv', header=0)
    centroids.index = [str(_) for _ in centroids.index]
    centroids.columns = ['Y','X']
    tmp = combine_adata_st[combine_adata_st.obs.layer == f'layer{layer}']
    tmp_index = tmp.obs.index
    tmp.obs.index=[_.split('-')[0] for _ in tmp.obs.index]
    combine_adata_st.obs['Y_pos'][tmp_index] = centroids.loc[tmp.obs.index]['Y']
    combine_adata_st.obs['X_pos'][tmp_index] = centroids.loc[tmp.obs.index]['X']

In [ ]:
from collections import Counter

ROI_cluster = pd.DataFrame(index=list(cluster_rough_dict.keys()))

combine_adata_st.obs['ROI'] = pd.Categorical(['other']*len(combine_adata_st), categories=list(ROI_mask.keys()) + ['other'], ordered=False)    
for _, mask in ROI_mask.items():
    yrange = mask.shape[0]
    for cell in tqdm(combine_adata_st[combine_adata_st.obs['leiden_type']!='-2'].obs.index, desc=_):
        if mask[yrange - max(1,int(float(combine_adata_st.obs['Y_pos'].loc[cell])/100)), int(max(0,float(combine_adata_st.obs['X_pos'].loc[cell])/100))]:
            combine_adata_st.obs['ROI'].loc[cell] = _
    
    tmp_adata_st = combine_adata_st[combine_adata_st.obs.ROI == _]
    tmp = Counter(tmp_adata_st[tmp_adata_st.obs.leiden_type != '-2'].obs.type)
    df = pd.DataFrame(tmp, index=[0]).T
    df.columns = [_]
    ROI_cluster = pd.concat([ROI_cluster, df], axis=1)

In [ ]:
from collections import Counter


ROI_cluster = pd.DataFrame(index=list(cluster_rough_dict.keys()))
for _, mask in ROI_mask.items():
    tmp_adata_st = combine_adata_st[combine_adata_st.obs.ROI == _]
    tmp = Counter(tmp_adata_st[tmp_adata_st.obs.leiden_type != '-2'].obs.type)
    df = pd.DataFrame(tmp, index=[0]).T
    df.columns = [_]
    ROI_cluster = pd.concat([ROI_cluster, df], axis=1)

In [ ]:
ROI_cluster_immune = ROI_cluster[ROI_cluster.index.isin(['Liver','Tumor','Endo','Ep','CAF'])]
ROI_cluster_nonimmune = ROI_cluster[~ROI_cluster.index.isin(['Liver','Tumor','Endo','Ep','CAF'])]

In [ ]:
import matplotlib.ticker as mtick
from matplotlib.colors import LinearSegmentedColormap


fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(30, 10))
# sc.pl.umap(
#     combine_adata_st[~combine_adata_st.obs['leiden_type'].isin(['-2','0'])],
#     size=1,
#     color="type",
#     palette=type_colormap,
#     # legend_loc="on data",
#     legend_fontweight=100,
#     legend_fontsize=10,
#     ax=ax[0],
#     show=False,
# )


colors = [_ for _ in type_colormap.values()][:-1]
cmap1 = LinearSegmentedColormap.from_list('my_colormap', colors[:5])
cmap2 = LinearSegmentedColormap.from_list('my_colormap', colors[5:])

df = ROI_cluster_immune.T
# df = df.drop('Liver', axis=1)
df = df.div(df.sum(axis=1), axis=0) * 100
df.plot(kind='bar', 
        stacked=True, 
        figsize=(20,10), 
        colormap=cmap1,
        ax=ax[0]
        )
ax[0].yaxis.set_major_formatter(mtick.PercentFormatter())
ax[0].set_title('content of different cluster on ROIs')
# plt.legend(loc='upper center',
#            bbox_to_anchor=(1.2, 1),
#            ncols=1,
#            )

df = ROI_cluster_nonimmune.T
# df = df.drop('Liver', axis=1)
df = df.div(df.sum(axis=1), axis=0) * 100
df.plot(kind='bar', 
        stacked=True, 
        figsize=(20,10), 
        colormap=cmap2,
        ax=ax[1]
        )
ax[1].yaxis.set_major_formatter(mtick.PercentFormatter())
ax[1].set_title('content of different cluster on ROIs')
# plt.legend(loc='upper center',
#            bbox_to_anchor=(1.2, 1),
#            ncols=1,
#            )
# plt.tight_layout()
plt.show()
# plt.savefig(r'E:\TMC\cell_typing\results\2023.9.28-_PRISM_HCC_final_downstream_analysis\2023.10.8_revised_Roe, imaris new color\4_ROI_fraction_type.pdf')

### Ro/e

In [ ]:
tmp = combine_adata_st.obs.subtype.replace({'T_CD8+':"T_CD8+_other","T_CD4+":"T_CD4+_other"})
combine_adata_st.obs.subtype = pd.Categorical(tmp, categories=tmp.unique())

In [ ]:
type_of_interest = [
    ["Tumor_AFP+"],
    ["Tumor_GPC3+"],
    ["Tumor_proliferation"],
    ["T_CD4+_other"],
    ["T_CD4+, PD1+"],
    ["T_CD4+, CXCL13+", "T_CD4+, PD1+, CXCL13+"],
    ["T_CD8+, GZMA+, CXCL13+"],
    ["T_CD4+, PD1+, CTLA4+"],
    ["Cyto_T_CD4+"],
    ["T_CD8+_other"],
    ["T_CD4+, CTLA4+"],
    ["T_CD8+, PD1+"],
    ["T_reg"],
    ["T_proliferation"],
    ["B_CD79A+"],
    ["B_MS4A1+"],
    ["NK_NCAM1+"],
    ["Endo_PECAM1+"],
    ["Ep_EPCAM+"],
    ["CAF_ACTA2+"],
    ]

ROIs = [f'ROI_{i}' for i in [1,
                             2,
                             4,3,5]]
R_oe = pd.DataFrame(columns=ROIs)
R_oe['type_of_in'] = [str(_) for _ in type_of_interest]
R_oe.set_index('type_of_in', inplace = True)

adata_for_ROE = combine_adata_st[combine_adata_st.obs.ROI.isin(ROIs)]
total_cell_num = len(adata_for_ROE)

for subtype in type_of_interest:
    # for region in ROI_mask.keys():
    for region in ROIs:
        adata_for_ROI = adata_for_ROE[adata_for_ROE.obs.ROI == region]
        adata_for_ROI_subtype = adata_for_ROI[adata_for_ROI.obs.subtype.isin(subtype)]

        observed_num = len(adata_for_ROI_subtype)
        expect_num = (len(adata_for_ROE[adata_for_ROE.obs.ROI==region]) * len(adata_for_ROE[adata_for_ROE.obs.subtype.isin(subtype)]))/total_cell_num
        # if expect_num == 0:
        #     R_oe.loc[str(subtype), region] = 0
        #     continue

        R_oe.loc[str(subtype), region] = observed_num/expect_num

In [ ]:
plt.figure(figsize=(10, 10))
sns.heatmap(
    [list(_) for _ in np.array(R_oe)],
    cmap="coolwarm",
    xticklabels=R_oe.columns,
    yticklabels=R_oe.index,
    annot=True,
    # vmax=2,
    # vmin=-0.5,
)
plt.title(f"ROE")
plt.show()
# plt.savefig(r'E:\TMC\cell_typing\results\2023.9.28-_PRISM_HCC_final_downstream_analysis\2023.10.8_revised_Roe, imaris new color\4_ROI_Roe.pdf')
# plt.yticks(np.arange(0.5, len(R_oe.index), 1), R_oe.index)
# plt.xticks(np.arange(0.5, len(R_oe.columns), 1), R_oe.columns)

## Type distribution on orientation

In [ ]:
import math
angle = math.atan(1/2)
direction_vector = np.array([math.cos(angle), math.sin(angle)])

xy_array = np.array([combine_adata_st.obs.X_pos, combine_adata_st.obs.Y_pos])
xy_array = xy_array.T

combine_adata_st.obs['direction_projection'] = xy_array@direction_vector

# type_of_interest_for_orientation = [
#     'CAF','Tumor','CD4+', 'CD8+', 'T_reg','NK', 'B', 'Monocyte','Neutrophil', 'Mait', 'Mast'
# ]

# subtype_of_interest_for_orientation = [
#     "Tumor_AFP+",
#     "Tumor_GPC3+",
#     "Tumor_proliferation",
#     "CAF_ACTA2+",
#     "Monocyte_CD14+",
#     "Monocyte_CD14+, CD16+",
#     "Monocyte_CD16+",
# ]

type_of_interest = [_ for _ in cluster_dict.keys()]
type_of_interest.remove('CAF_ACTA2+')
type_of_interest = ['CAF_ACTA2+'] + type_of_interest


fig, ax = plt.subplots(ncols=1,nrows=len(type_of_interest),figsize=(5,2*(len(type_of_interest))))
for _, subtype in enumerate(type_of_interest):
    a=combine_adata_st[combine_adata_st.obs.subtype==subtype].obs['direction_projection']
    sns.histplot(a, bins=100, stat='density', alpha=1, kde=True,
                edgecolor='white', linewidth=0.5,
                color=subtype_colormap[subtype],
                ax=ax[_],
                line_kws=dict(color='black', alpha=0.7, linewidth=1.5, label='KDE'),
                # binrange=[0,100]
                )
    ax[_].set_xlim(0,55000)
    ax[_].legend([subtype])
    if subtype == 'CAF_ACTA2+':
        y = ax[_].get_lines()[0].get_ydata()
        maxima = [float(_/len(y)*(max(a)-min(a))+min(a)) for _ in argrelextrema(-np.array(y), np.less)[0]]
    for submaxima in maxima:
        ax[_].axvline(x=submaxima, color='r', alpha=0.5, linestyle='--')

# for _, subtype in enumerate(subtype_of_interest_for_orientation):
#     _ += len(type_of_interest_for_orientation)
#     a=combine_adata_st[combine_adata_st.obs.subtype==subtype].obs['direction_projection']
#     sns.histplot(a, bins=100, stat='density', alpha=1, kde=True,
#                 edgecolor='white', linewidth=0.5,
#                 # log=True,
#                 ax=ax[_],
#                 line_kws=dict(color='black', alpha=0.7, linewidth=1.5, label='KDE'),
#                 # binrange=[0,100]
#                 )
#     ax[_].set_xlim(0,55000)
#     ax[_].legend([subtype])
#     # y.append(ax[_].get_lines()[0].get_ydata())

#     # if subtype == 'CAF':
#     #     maxima = [float(j/len(y[_])*(max(a)-min(a))+min(a)) for j in argrelextrema(-np.array(y[_]), np.less)[0]]

#     for submaxima in maxima:
#         ax[_].axvline(x=submaxima, color='r', alpha=0.5, linestyle='--')

plt.suptitle('angle={0:.2f}pi'.format(angle/math.pi))
plt.tight_layout()
# plt.savefig(r'E:\TMC\cell_typing\results\2023.9.28-_PRISM_HCC_final_downstream_analysis\figures\4_distribution_sep_by_CAF_all.pdf', bbox_inches = 'tight')
plt.show()

In [ ]:
import io
import PySimpleGUI as sg
from PIL import Image, ImageDraw, ImageTk

# Function to convert the PIL image to a format that PySimpleGUI can display
def convert_to_bytes(image):
    with io.BytesIO() as buffer:
        image.save(buffer, format="PNG")
        return buffer.getvalue()

# Load the initial image and create a drawing object
image_path = './dataset/PRISM30_mousebrain/figures/layer1.jpg'  # Replace with your image path
original_image = Image.open(image_path)
canvas_size = original_image.size

# Create a separate mask image (initially transparent)
mask = Image.new("RGBA", canvas_size, (0, 0, 0, 0))
draw = ImageDraw.Draw(mask)

# Define the tab layout with a Canvas element
tab_layout = [
    [sg.Canvas(size=canvas_size, key='-CANVAS-')],
    [sg.Button('Save', key='-SAVE-'), sg.Button('Exit', key='-EXIT-')]
]

# Create the main window layout with tabs
layout = [
    [sg.TabGroup([[sg.Tab('Drawing Tab', tab_layout)]])],
]

# Create the window
window = sg.Window('Draw Semi-Transparent Mask', layout, finalize=True)
canvas = window['-CANVAS-'].TKCanvas

# Function to update the canvas with the blended image
def update_canvas():
    global tk_image
    canvas.delete("all")  # Clear the current image
    # Blend the mask with the original image
    blended_image = Image.alpha_composite(original_image.convert("RGBA"), mask)
    tk_image = ImageTk.PhotoImage(blended_image)
    canvas.create_image(0, 0, image=tk_image, anchor='nw')

# Draw the initial image on the canvas
update_canvas()

# Function to handle drawing
def draw_circle(event):
    x, y = event.x, event.y
    radius = 10  # Adjust radius as needed
    draw.ellipse((x-radius, y-radius, x+radius, y+radius), fill=(255, 0, 0, 128))  # Semi-transparent red
    update_canvas()

# Bind mouse events to the drawing function
canvas.bind('<B1-Motion>', draw_circle)  # Draw on drag

# Event loop
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event == '-EXIT-':
        break
    elif event == '-SAVE-':
        mask.save('path_to_save_mask.png')  # Replace with desired save path for mask

window.close()


In [ ]:
import math
angle = math.atan(1/2)
direction_vector = np.array([math.cos(angle), math.sin(angle)])

xy_array = np.array([combine_adata_st.obs.X_pos, combine_adata_st.obs.Y_pos])
xy_array = xy_array.T

combine_adata_st.obs['direction_projection'] = xy_array@direction_vector

type_of_interest_for_orientation = [
    "Tumor_AFP+",
    "Tumor_GPC3+",
    "Tumor_proliferation",
    "CAF_ACTA2+",
    "Monocyte_CD14+",
    "Monocyte_CD14+, CD16+",
    "Monocyte_CD16+",
]

y=[]

fig, ax = plt.subplots(ncols=1,nrows=len(type_of_interest_for_orientation),figsize=(5,2*len(type_of_interest_for_orientation)))
for _, subtype in enumerate(type_of_interest_for_orientation):
    a=combine_adata_st[combine_adata_st.obs.subtype==subtype].obs['direction_projection']
    sns.histplot(a, bins=100, stat='density', alpha=1, kde=True,
                edgecolor='white', linewidth=0.5,
                # log=True,
                ax=ax[_],
                line_kws=dict(color='black', alpha=0.7, linewidth=1.5, label='KDE'),
                # binrange=[0,100]
                )
    ax[_].set_xlim(0,55000)
    ax[_].legend([subtype])
    y.append(ax[_].get_lines()[0].get_ydata())

    if subtype == 'CAF':
        maxima = [float(j/len(y[_])*(max(a)-min(a))+min(a)) for j in argrelextrema(-np.array(y[_]), np.less)[0]]

    for submaxima in maxima:
        ax[_].axvline(x=submaxima, color='r', alpha=0.5, linestyle='--')
    
plt.suptitle('angle={0:.2f}pi'.format(angle/math.pi))
plt.tight_layout()
# plt.show()
plt.savefig(r'E:\TMC\cell_typing\results\2023.9.28-_PRISM_HCC_final_downstream_analysis\figures\4_distribution_sep_by_CAF_TOI.pdf')

print(f'maxima: {maxima}')

In [ ]:
fig,ax = plt.subplots(ncols=1, nrows=1, figsize=(20,10))
for line in y:
    ax = plt.plot(line)

plt.legend(type_of_interest_for_orientation)
plt.show
# plt.savefig(r'E:\TMC\cell_typing\results\2023.9.28-_PRISM_HCC_final_downstream_analysis\figures\4_distribution_sep_by_CAF_smoothed.pdf')

# Plot4: quantitative spatial analysis

In [ ]:
import squidpy as sq

In [ ]:
combine_adata_st.obsm['spatial3d'] = np.array([combine_adata_st.obs.X_pos,combine_adata_st.obs.Y_pos,[int(_.replace('layer',''))*10/0.1625 for _ in combine_adata_st.obs.layer]]).T

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
sq.pl.spatial_scatter(combine_adata_st, shape=None, color="type", size=0.1, ax=ax)
ax.invert_yaxis()
plt.show()

In [ ]:
sc.pl.embedding(combine_adata_st, basis="spatial", projection="3d", color="type", palette=type_colormap, size=0.1)

In [ ]:
sq.gr.spatial_neighbors(combine_adata_st, coord_type="generic", spatial_key="spatial3d")

## Plot: spatial neighborhood enrichment

In [ ]:
combine_adata_st = sc.read_h5ad(cell_typ_dir/'adata_PRISM_HCC_3D_pseudo_harmony.h5ad')

In [ ]:
type_reorder = ['Tumor', 'Liver', 'DC', 'T_proliferation', 'Mast', 'CAF', 'Endo', 'Neutrophil', 'CD8+', 'NK', 'Monocyte', 'CD4+', 'Mait', 'Macrophage', 'T_reg', 'B', 'Ep', 'other']

tmp = combine_adata_st.copy()
tmp.obs.type = pd.Categorical(tmp.obs.type, categories=type_reorder)
tmp.obs.subtype = tmp.obs.subtype.replace({'T_CD4+, PD1+, CXCL13+':'T_CD4+, CXCL13+'})
tmp.obs.subtype = pd.Categorical(tmp.obs.subtype, categories=tmp.obs.subtype.unique())

In [ ]:
sq.gr.nhood_enrichment(tmp, cluster_key="type")
sq.gr.nhood_enrichment(tmp, cluster_key="subtype")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sq.pl.nhood_enrichment(tmp, cluster_key="type", cmap="coolwarm",
                       vmin=-100, vmax=100, ax=ax)
plt.show()
# plt.savefig(r'E:\TMC\cell_typing\results\2023.9.28-_PRISM_HCC_final_downstream_analysis\figures\neighbor.pdf')

In [ ]:
tmp = combine_adata_st.copy()
subtype_reorder = [
    'Liver_normal',
    'Tumor_AFP+',
    'Tumor_GPC3+',
    'Tumor_proliferation',
    'Endo_PECAM1+',
    'Ep_EPCAM+',
    'CAF_ACTA2+',
    'T_reg',
    'T_proliferation',
    "T_CD4+_other",
    
    "T_CD4+, CTLA4+",
    "T_CD4+, PD1+, CTLA4+",
   
    "T_CD4+, PD1+",
    "T_CD4+, CXCL13+",
    "T_CD8+, GZMA+, CXCL13+",
    "T_CD8+, PD1+",

    "T_CD8+_other",
    
    "Cyto_T_CD4+",
    'NK_NCAM1+',
    
    'B_CD79A+',
    'B_MS4A1+',
    
    'pDC_LILRA4+',
    'cDC1_CLEC9A+',
    'cDC2_CD1C+',
    
    'Macrophage_LYVE1+',
    
    'Monocyte_CD16+',
    'Monocyte_CD14+, CD16+',
    'Monocyte_CD14+',

    'Neutrophil_CSF3R+, S100A8+',
    'Neutrophil_CSF3R+',
    
    'Mait_SLC4A10+',
    'Mast_CPA3+',
    'other',
    ]

tmp.obs.subtype = tmp.obs.subtype.replace({'T_CD4+, PD1+, CXCL13+':'T_CD4+, CXCL13+'})
tmp.obs.subtype = pd.Categorical(tmp.obs.subtype, categories=subtype_reorder)

In [ ]:
sq.gr.nhood_enrichment(tmp, cluster_key="subtype")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sq.pl.nhood_enrichment(tmp, cluster_key="subtype", method='ward', cmap="coolwarm",
                       vmin=-100, vmax=100, ax=ax)
plt.show()

### validation by projection

In [ ]:
tmp.obsm['spatial'].max(axis=0)

In [ ]:
from spatial import downsize_to_tif


neighbor_type_toplot = ['Tumor_AFP+',]
for subtype in neighbor_type_toplot:
    points = tmp[tmp.obs.subtype == subtype].obsm['spatial']
    downsize_to_tif(points=points, max_point_values=[1169.23076923, 38920.96604114, 47526.29253084,],
                    out_path=cell_typ_dir/f'neighbor_enrichment_projection_{subtype}.tif', binsize=200)

## Plot: spatial auto correlation

In [ ]:
sq.gr.co_occurrence(tmp, cluster_key="subtype")

In [ ]:
sq.pl.co_occurrence(tmp, cluster_key="subtype", clusters="Tumor_AFP+")

## Plot interaction matrix

In [ ]:
sq.gr.interaction_matrix(tmp, cluster_key="subtype")

In [ ]:
sq.pl.interaction_matrix(tmp, cluster_key="subtype",
                         vmin=0, vmax=50000, method='centroid')

## Plot gene autocorrelation

In [ ]:
tmp_slice = tmp[tmp.obs.layer=='layer10']
sq.gr.spatial_autocorr(tmp_slice, mode="moran")
tmp_slice.uns["moranI"].head(10)

# Plot5: heatmap

## cluster of sc data

In [ ]:
# adata_sc_subset = adata_sc1_subset.concatenate(adata_sc2_subset, adata_sc3_subset, batch_key="dataset", batch_categories=["GSE151530", "GSE140228", "CNP0000650"])
# adata_sc_subset = combine_adata[combine_adata.obs.dataset.isin(['GSE151530','GSE140228','CNP0000650'])]

In [ ]:
import scanpy.external as sce


adata_sc_subset = adata_sc_subset[:, list_of_variable_names]
sc.tl.pca(adata_sc_subset, n_comps=29)
sc.pl.pca_variance_ratio(adata_sc_subset, log=False)

h_pcs = 29
sc.tl.pca(adata_sc_subset, n_comps=h_pcs)
print(adata_sc_subset)

sce.pp.harmony_integrate(adata_sc_subset, "dataset",
                         "X_pca", "X_pca_harmony", max_iter_harmony=30)
neighbor = 50
sc.pp.neighbors(adata_sc_subset, n_neighbors=neighbor, use_rep="X_pca_harmony")
sc.tl.umap(adata_sc_subset)

In [ ]:
leiden_resolution=1
sc.tl.leiden(adata_sc_subset, resolution=leiden_resolution)

In [ ]:
UMAP_genes_plot(adata=adata_sc_subset, size=1, save=False, out_path=cell_typ_dir)
UMAP_obs_plot(adata=adata_sc_subset, color='leiden', save=False, out_path=cell_typ_dir, datasets=['GSE151530','GSE140228','CNP0000650'])
sc.pl.dotplot(adata_sc_subset, var_names=adata_sc_subset.var.index, groupby='leiden')

## calculate of coor matrix

In [ ]:
combine_adata_st = sc.read_h5ad(r'e:\TMC\cell_typing\results\2023.9.28-_PRISM_HCC_final_downstream_analysis\2023.10.3_20layers_corr&neighbor_enrichment\adata_neighborhood_enrichment.h5ad')
adata_sc_subset = sc.read_h5ad(r'E:\TMC\cell_typing\results\2023.8.21-_PRISM_HCC_one_layer\2023.10.12_leiden_final_typed\adata_sc_subset_retyped.h5ad')
adata = sc.read_h5ad(r'E:\TMC\cell_typing\results\2023.9.19-9.22_PRISM_HCC_20_layers_min_counts=7, max_counts=200, min_genes=2\direct\adata_temp.h5ad')

In [ ]:
adata_sc_subset = sc.read_h5ad(r'E:\TMC\cell_typing\results\2023.8.21-_PRISM_HCC_one_layer\2023.10.12_leiden_final_typed\adata_sc_subset_res=4,retyped_by_zch.h5ad')

In [ ]:
cluster_of_intere = list(cluster_rough_dict.keys())
raw_data_matrix, sc_data_matrix, corr_matrix = matrix_for_heatmap(
    combine_adata_st, 
    adata_sc_subset, 
    combine_adata_st, 
    obs_1="type", 
    obs_2="type", 
    cluster_of_intere=cluster_of_intere, 
    sc_cluster_of_intere=cluster_of_intere, 
    save=False, 
    whole=False
    )

## plot heatmaps

### exp data

In [ ]:
raw_plot = raw_data_matrix.copy()
for i in range(raw_plot.shape[1]):
    raw_plot[:, i] = raw_plot[:, i] / np.linalg.norm(raw_plot[:, i])
# for i in range(raw_plot.shape[0]):
#     raw_plot[i, :] = raw_plot[i, :] / np.linalg.norm(raw_plot[i, :])

plt.figure(figsize=(raw_plot.shape[1], raw_plot.shape[0]))
sns.heatmap(
    raw_plot,
    cmap="coolwarm",
    annot=True,
    xticklabels=list(adata.var_names),
    # yticklabels=ystick,
    # vmax=2, vmin=-0.5,
)
plt.title(f"PRISM_raw_cluster_gene_mean_expression_heatmap")
# np.savetxt(f"{cell_typ_dir}/PRISM_raw_cluster_gene_mean_expression_heatmap.csv", raw_plot,delimiter=",")
# plt.savefig(f"{cell_typ_dir}/PRISM_raw_cluster_gene_mean_expression_heatmap.pdf")
plt.show()

### sc data

In [ ]:
sc_plot = sc_data_matrix.copy()
for i in range(sc_plot.shape[0]):
    sc_plot[i, :] = sc_plot[i, :] / np.linalg.norm(sc_plot[i, :])

# for i in range(sc_plot.shape[1]):
#     sc_plot[:, i] = sc_plot[:, i] / np.linalg.norm(sc_plot[:, i])

plt.figure(figsize=(sc_plot.shape[1], sc_plot.shape[0]))
sns.heatmap(
    sc_plot,
    cmap="coolwarm",
    annot=True,
    # yticklabels=sc_clus_dict.keys(),
    xticklabels=list(adata_sc_subset.var_names),
)
plt.title(f"PRISM_sc_cluster_gene_mean_expression_heatmap")

np.savetxt(cell_typ_dir/"PRISM_sc_cluster_gene_mean_expression_heatmap.csv", sc_plot, delimiter=",")
plt.savefig(cell_typ_dir/"PRISM_sc_cluster_gene_mean_expression_heatmap.pdf")
plt.show()

### exp_data vs. sc_data

In [ ]:
# map_plot of raw vs. sc cluster
map_plot = pd.DataFrame(corr_matrix, columns=cluster_of_intere, index=cluster_of_intere)
df = map_plot.copy()
# all_nan_columns = df.columns[df.isna().all()].tolist()
# all_nan_rows = df.index[df.isna().all(axis=1)].tolist()
# union_set = set(all_nan_columns).union(all_nan_rows)
# df = df.drop(union_set, axis=0)
# df = df.drop(union_set, axis=1)


plt.figure(figsize=(24, 18))
heatmap = sns.heatmap(
    df,
    cmap="coolwarm",
    annot=True,
    # yticklabels=cluster_of_intere,
    # xticklabels=cluster_of_intere,
    # vmin=0,
    # vmax=0.6,
)
# plt.show()
df = pd.DataFrame(map_plot, columns=cluster_of_intere, index=cluster_of_intere)
df.to_csv(r'E:\TMC\cell_typing\results\2023.9.28-_PRISM_HCC_final_downstream_analysis\figures\5_vs_subtype_heatmap_sc_retyped_by_zch_dropna_reordered.csv')
plt.savefig(r'E:\TMC\cell_typing\results\2023.9.28-_PRISM_HCC_final_downstream_analysis\figures\5_vs_subtype_heatmap_sc_retyped_by_zch_dropna_reordered.pdf', bbox_inches = 'tight')